In [1]:
import pandas as pd
import numpy as np

In [2]:
flights = pd.read_csv('data/flights.csv')
flights.head()

,MONTH,DAY,WEEKDAY,AIRLINE,ORG_AIR,DEST_AIR,SCHED_DEP,DEP_DELAY,AIR_TIME,DIST,SCHED_ARR,ARR_DELAY,DIVERTED,CANCELLED
0,1,1,4,WN,LAX,SLC,1625,58.0,94.0,590,1905,65.0,0,0
1,1,1,4,UA,DEN,IAD,823,7.0,154.0,1452,1333,-13.0,0,0
2,1,1,4,MQ,DFW,VPS,1305,36.0,85.0,641,1453,35.0,0,0
3,1,1,4,AA,DFW,DCA,1555,7.0,126.0,1192,1935,-7.0,0,0
4,1,1,4,WN,LAX,MCI,1720,48.0,166.0,1363,2225,39.0,0,0


In [3]:
fp = flights.pivot_table(index='AIRLINE', 
                         columns='ORG_AIR', 
                         values='CANCELLED', 
                         aggfunc='sum',
                         fill_value=0).round(2)
fp.head()

ORG_AIR,ATL,DEN,DFW,IAH,LAS,LAX,MSP,ORD,PHX,SFO
AIRLINE,,,,,,,,,,
AA,3,4,86,3,3,11,3,35,4,2
AS,0,0,0,0,0,0,0,0,0,0
B6,0,0,0,0,0,0,0,0,0,1
DL,28,1,0,0,1,1,4,0,1,2
EV,18,6,27,36,0,0,6,53,0,0


In [4]:
fg = flights.groupby(['AIRLINE', 'ORG_AIR'])['CANCELLED'].sum()
fg.head()

AIRLINE  ORG_AIR
AA       ATL         3
         DEN         4
         DFW        86
         IAH         3
         LAS         3
Name: CANCELLED, dtype: int64

In [5]:
fg_unstack = fg.unstack('ORG_AIR', fill_value=0)
fg_unstack.head()

ORG_AIR,ATL,DEN,DFW,IAH,LAS,LAX,MSP,ORD,PHX,SFO
AIRLINE,,,,,,,,,,
AA,3,4,86,3,3,11,3,35,4,2
AS,0,0,0,0,0,0,0,0,0,0
B6,0,0,0,0,0,0,0,0,0,1
DL,28,1,0,0,1,1,4,0,1,2
EV,18,6,27,36,0,0,6,53,0,0


In [6]:
fp.equals(fg_unstack)

True

In [7]:
fp2 = flights.pivot_table(index=['AIRLINE', 'MONTH'],
                          columns=['ORG_AIR', 'CANCELLED'],
                          values=['DEP_DELAY', 'DIST'],
                          aggfunc=[np.mean, np.sum],
                          fill_value=0)
fp2.head()

mean                                                 \
              DEP_DELAY                                                  
ORG_AIR             ATL           DEN           DFW             IAH      
CANCELLED             0  1          0  1          0    1          0  1   
AIRLINE MONTH                                                            
AA      1     -3.250000  0   7.062500  0  11.977591 -3.0   9.750000  0   
        2     -3.000000  0   5.461538  0   8.756579  0.0   1.000000  0   
        3     -0.166667  0   7.666667  0  15.383784  0.0  10.900000  0   
        4      0.071429  0  20.266667  0  10.501493  0.0   6.933333  0   
        5      5.777778  0  23.466667  0  16.798780  0.0   3.055556  0   

                             ...      sum                                \
                             ...     DIST                                 
ORG_AIR              LAS     ...      LAX          MSP        ORD         
CANCELLED              0  1  ...        0     1      0  1       0     1   
AIRLINE MONTH                ...                                          
AA      1      32.375000  0  ...   135921  2475   7281  0  129334     0   
        2      -3.055556  0  ...   113483  5454   5040  0  120572  5398   
        3      12.074074  0  ...   131836  1744  14471  0  127072   802   
        4      27.241379  0  ...   170285     0   4541  0  152154  4718   
        5       2.818182  0  ...   167484     0   6298  0  110864  1999   

                                        
                                        
ORG_AIR          PHX         SFO        
CANCELLED          0    1      0     1  
AIRLINE MONTH                           
AA      1      21018    0  33483     0  
        2      17049  868  32110  2586  
        3      25770    0  43580     0  
        4      17727    0  51054     0  
        5      11164    0  40233     0  

[5 rows x 80 columns]

In [8]:
flights.groupby(['AIRLINE', 'MONTH', 'ORG_AIR', 'CANCELLED'])['DEP_DELAY', 'DIST'] \
       .agg(['mean', 'sum']) \
       .unstack(['ORG_AIR', 'CANCELLED'], fill_value=0) \
       .swaplevel(0, 1, axis='columns') \
       .head()

mean                                                    \
              DEP_DELAY                                                     
ORG_AIR             ATL            DEN            DFW             IAH       
CANCELLED             0   1          0   1          0    1          0   1   
AIRLINE MONTH                                                               
AA      1     -3.250000 NaN   7.062500 NaN  11.977591 -3.0   9.750000 NaN   
        2     -3.000000 NaN   5.461538 NaN   8.756579  NaN   1.000000 NaN   
        3     -0.166667 NaN   7.666667 NaN  15.383784  NaN  10.900000 NaN   
        4      0.071429 NaN  20.266667 NaN  10.501493  NaN   6.933333 NaN   
        5      5.777778 NaN  23.466667 NaN  16.798780  NaN   3.055556 NaN   

                               ...         sum                                 \
                               ...        DIST                                  
ORG_AIR              LAS       ...         LAX              MSP           ORD   
CANCELLED              0   1   ...           0       1        0   1         0   
AIRLINE MONTH                  ...                                              
AA      1      32.375000 NaN   ...    135921.0  2475.0   7281.0 NaN  129334.0   
        2      -3.055556 NaN   ...    113483.0  5454.0   5040.0 NaN  120572.0   
        3      12.074074 NaN   ...    131836.0  1744.0  14471.0 NaN  127072.0   
        4      27.241379 NaN   ...    170285.0     NaN   4541.0 NaN  152154.0   
        5       2.818182 NaN   ...    167484.0     NaN   6298.0 NaN  110864.0   

                                                        
                                                        
ORG_AIR                    PHX             SFO          
CANCELLED           1        0      1        0       1  
AIRLINE MONTH                                           
AA      1         NaN  21018.0    NaN  33483.0     NaN  
        2      5398.0  17049.0  868.0  32110.0  2586.0  
        3       802.0  25770.0    NaN  43580.0     NaN  
        4      4718.0  17727.0    NaN  51054.0     NaN  
        5      1999.0  11164.0    NaN  40233.0     NaN  

[5 rows x 80 columns]